# Installing Packages

In [1]:
%%capture
!pip install pip3-autoremove
!pip-autoremove torch torchvision torchaudio -y
!pip install "torch==2.4.0" "xformers==0.0.27.post2" triton torchvision torchaudio "trl==0.14.0"
!pip install "unsloth[kaggle-new] @ git+https://github.com/unslothai/unsloth.git"
!pip uninstall -y trl
!pip install git+https://github.com/huggingface/trl.git@e95f9fb74a3c3647b86f251b7e230ec51c64b72b

# Import libs

In [2]:
import numpy as np
import pandas as pd
import json

import torch
import torch.nn.functional as F
from unsloth import FastLanguageModel
from datasets import Dataset, load_dataset

import os
from tqdm.auto import tqdm
from IPython.display import clear_output

os.environ["WANDB_DISABLED"] = "true"
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

Unsloth: Patching Xformers to fix some performance issues.
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-06-25 15:51:22.711949: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750866682.891181      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750866682.946085      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!


# Initializing Model

- Quantization: 4bit
- Flash Attention 2

In [3]:
model_name = "unsloth/Qwen2.5-0.5B-Instruct-bnb-4bit"

# >>> Download LLM Model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = 8192,
    dtype = None,
    load_in_4bit = True,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

# FastLanguageModel.for_inference(model)
# print("Model is ready to inference

==((====))==  Unsloth 2025.6.5: Fast Qwen2 patching. Transformers: 4.51.3.
   \\   /|    Tesla P100-PCIE-16GB. Num GPUs = 1. Max memory: 15.888 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.4.0+cu121. CUDA: 6.0. CUDA Toolkit: 12.1. Triton: 3.0.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/457M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

- LoRA (Low-Rank Adaptation) with r = 128, alpha = 16, dropout = 0
- target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.6.5 patched 24 layers with 24 QKV layers, 24 O layers and 24 MLP layers.


# Load & process dataset

- Data Source: https://huggingface.co/datasets/nhantruongcse/summary-vietnamese-news

In [5]:
ds = load_dataset("nhantruongcse/summary-vietnamese-news")["train"]

README.md:   0%|          | 0.00/321 [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/165M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/161M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/164573 [00:00<?, ? examples/s]

In [6]:
ds

Dataset({
    features: ['Content', 'Summary'],
    num_rows: 164573
})

In [7]:
system_message = """Bạn là trợ lý AI có nhiệm vụ tóm tắt văn bản của người dùng cung cấp.

### Hướng dẫn trả lời:
Không dài dòng, giới hạn 100 từ
Không giải thích gì thêm, chỉ cần tóm tắt
"""

user_template = """Hãy tóm tắt văn bản dưới đây:

{}
"""

assistant_template = """{}
"""

def apply_template_message(data):
    messages = [{
        "role": "system",
        "content": system_message
    }, {
        "role": "user",
        "content": user_template.format(data["Content"])
    }, {
        "role": "assistant",
        "content": assistant_template.format(data["Summary"])
    }]
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False, enable_thinking=False)
    data["text"] = text
    return data

def apply_template_message_eval(data):
    messages = [{
        "role": "system",
        "content": system_message
    }, {
        "role": "user",
        "content": user_template.format(data["Content"])
    }]
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True, e=False)
    data["text"] = text
    return data

In [8]:
split_dataset = ds.train_test_split(test_size=0.2, seed=42)

train_ds = split_dataset["train"].select(range(10000)) # Just take 10.000 samples
test_ds = split_dataset["test"].select(range(2000)) # Just take 2.000

In [9]:
train_ds = train_ds.map(apply_template_message)
test_ds = test_ds.map(apply_template_message_eval)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [10]:
print(train_ds[0]["text"])

<|im_start|>system
Bạn là trợ lý AI có nhiệm vụ tóm tắt văn bản của người dùng cung cấp.

### Hướng dẫn trả lời:
Không dài dòng, giới hạn 100 từ
Không giải thích gì thêm, chỉ cần tóm tắt
<|im_end|>
<|im_start|>user
Hãy tóm tắt văn bản dưới đây:

Thông báo kết luận của Thủ tướng tại cuộc họp Thường trực Chính phủ về dự án thép Hoa Sen Cà Ná đánh giá, dự án mới dừng ở mức đánh giá sơ bộ, công tác chuẩn bị dự án còn vội vàng, thông tin về dự án còn bất cập, chưa đầy đủ, toàn diện. Do đó, Thủ tướng đề nghị tỉnh Ninh Thuận và các cơ quan tạm dừng đề xuất dự án này để làm rõ một số vấn đề. Thứ nhất là phải tính toán kỹ nhu cầu thị trường thép trong nước và thế giới, trên cơ sở đó rà soát quy hoạch các nhà máy thép, cơ cấu sản phẩm theo nhu cầu thị trường để xác định quy mô, công suất và thời điểm hợp lý mới phát triển dự án. Thứ hai là phải đánh giá kỹ vấn đề môi trường, công nghệ và thiết bị của dự án, đặc biệt nghiên cứu đánh giá tác động môi trường đảm bảo dự án an toàn không xảy ra sự cố

In [11]:
print(test_ds[0]["text"])

<|im_start|>system
Bạn là trợ lý AI có nhiệm vụ tóm tắt văn bản của người dùng cung cấp.

### Hướng dẫn trả lời:
Không dài dòng, giới hạn 100 từ
Không giải thích gì thêm, chỉ cần tóm tắt
<|im_end|>
<|im_start|>user
Hãy tóm tắt văn bản dưới đây:

Huỳnh Như (9) và Tuyết Dung (7) là cả kho tàng kinh nghiệm để các cầu thủ trẻ dựa vào Ngọc Dương Thanh Nhã được AFC ca ngợi, vào top 6 ngôi sao trẻ hay nhất châu Á tại World Cup Trong năm 2023, đội tuyển nữ Việt Nam và thủ quân đón nhiều tín hiệu tích cực từ các cầu thủ trẻ. Vạn Sự, Hải Linh, Vũ Thị Hoa hay Thanh Nhã liên tục thể hiện được tiến bộ. Họ được trao cơ hội nhiều hơn, chơi ổn định, thậm chí là tỏa sáng ở SEA Games 32 và chạy đà cho World Cup 2023. Thậm chí Thanh Nhã, Hải Linh còn đủ sức chen chân vào đội hình chính ở SEA Games 32. Tuy nhiên, để có thể chơi tốt ở World Cup 2023, những "búp măng" này cần sự dìu dắt của các đàn chị. Điều đương nhiên phải nhắc đến là giá trị về chuyên môn. Thời điểm này Huỳnh Như, Tuyết Dung, Thùy Trang,

# Finetuning model

In [12]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_ds,
    dataset_text_field = "text",
    max_seq_length = 8092,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 8,
        gradient_accumulation_steps = 4, 
        warmup_steps = 5,
        num_train_epochs = 1,
        # max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 30,
        
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
    ),
)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [13]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 10,000 | Num Epochs = 1 | Total steps = 312
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 4 x 1) = 32
 "-____-"     Trainable parameters = 17,596,416/500,000,000 (3.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
30,2.493200
60,2.292400
90,2.273000
120,2.253500
150,2.237000
180,2.258400
210,2.231500
240,2.242900
270,2.233000
300,2.254700


# Save model

In [14]:
model.save_pretrained("qwen_finetuned_adapter")
tokenizer.save_pretrained("qwen_finetuned_adapter")

('qwen_finetuned_adapter/tokenizer_config.json',
 'qwen_finetuned_adapter/special_tokens_map.json',
 'qwen_finetuned_adapter/vocab.json',
 'qwen_finetuned_adapter/merges.txt',
 'qwen_finetuned_adapter/added_tokens.json',
 'qwen_finetuned_adapter/tokenizer.json')